## 第一部分：了解 nn.Module的基本操作

In [1]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F
import numpy as np

In [3]:
model = models.resnet18()

### 打印出 model底下所有 parameters 的 name 以及對應的 shape 

In [6]:
for name, param in model.named_parameters():
    print(name, param.shape)

conv1.weight torch.Size([64, 3, 7, 7])
bn1.weight torch.Size([64])
bn1.bias torch.Size([64])
layer1.0.conv1.weight torch.Size([64, 64, 3, 3])
layer1.0.bn1.weight torch.Size([64])
layer1.0.bn1.bias torch.Size([64])
layer1.0.conv2.weight torch.Size([64, 64, 3, 3])
layer1.0.bn2.weight torch.Size([64])
layer1.0.bn2.bias torch.Size([64])
layer1.1.conv1.weight torch.Size([64, 64, 3, 3])
layer1.1.bn1.weight torch.Size([64])
layer1.1.bn1.bias torch.Size([64])
layer1.1.conv2.weight torch.Size([64, 64, 3, 3])
layer1.1.bn2.weight torch.Size([64])
layer1.1.bn2.bias torch.Size([64])
layer2.0.conv1.weight torch.Size([128, 64, 3, 3])
layer2.0.bn1.weight torch.Size([128])
layer2.0.bn1.bias torch.Size([128])
layer2.0.conv2.weight torch.Size([128, 128, 3, 3])
layer2.0.bn2.weight torch.Size([128])
layer2.0.bn2.bias torch.Size([128])
layer2.0.downsample.0.weight torch.Size([128, 64, 1, 1])
layer2.0.downsample.1.weight torch.Size([128])
layer2.0.downsample.1.bias torch.Size([128])
layer2.1.conv1.weight tor

### 為了使 forward propagation 加速 並降低 memory 使用量，請將所有 parameters 的requires_grad 關閉，關閉之後執行  forward propagation

In [7]:
input_ = torch.randn(1, 3, 128, 128)

In [8]:
for param in model.parameters():
    param.requires_grad = False

In [9]:
output = model(input_)
print(output.shape)

torch.Size([1, 1000])


## 第二部分：依照指令，以較簡潔的方式搭建出模型

* input_shape = torch.Size([10, 12])
* 先經過一層 nn.Linear(12, 10)
* 經過10層 nn.Linear(10, 10)
* 最後經過 nn.Linear(10, 3) 輸出
* 每一個 nn.Linear過完後要先經過 nn.BatchNorm1d 才能到下一層，輸出層不用


In [12]:
input_ = torch.randn(10,12)
## 示範
Linear_0 = nn.Linear(12,10)
Linear = nn.Linear(10,10)
Linear_output = nn.Linear(10,3)
BN = nn.BatchNorm1d(10)

In [13]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.linear_0 = Linear_0
        self.linear_block = nn.Sequential(
            Linear,
            BN
        )
        self.linear_output = Linear_output

    def forward(self, x):
        x = self.linear_0(x)
        for i in range(10):
          x = self.linear_block(x)
        x = self.linear_output(x)
        return x

In [14]:
model = Model()

In [15]:
model

Model(
  (linear_0): Linear(in_features=12, out_features=10, bias=True)
  (linear_block): Sequential(
    (0): Linear(in_features=10, out_features=10, bias=True)
    (1): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (linear_output): Linear(in_features=10, out_features=3, bias=True)
)

In [18]:
input_ = torch.randn(10,12)
outupt = model(input_)

In [19]:
outupt

tensor([[ 0.0331,  0.6247, -0.3711],
        [-0.3505, -1.0824, -0.1226],
        [-0.3709,  0.3181, -0.5642],
        [-0.2115, -0.6034, -1.9978],
        [-0.6029,  0.1875, -0.3211],
        [-0.2578, -0.5542, -0.3779],
        [-0.3527, -0.7448,  0.7331],
        [ 0.0514, -1.2274,  0.1230],
        [-0.3554,  0.2408,  0.0192],
        [ 0.1336,  0.6462,  0.2224]], grad_fn=<AddmmBackward>)